# Imports

In [ ]:
import os
# os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID" 
# os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
# Uncomment the above 2 lines if you want to use your CPU instead of GPU

import time
import numpy as np
import sys
import gc

import keras
print("Keras version:", keras.__version__)

import tensorflow as tf
print("TensorFlow version:", tf.__version__)

from PIL import ImageFile
# the following will allow "damaged" image files to be loaded without an exception firing
ImageFile.LOAD_TRUNCATED_IMAGES = True 

from keras.models import Model
from keras import layers
from keras import optimizers
from keras.layers.convolutional import Convolution2D
from keras.layers.pooling import MaxPooling2D
from keras.layers.pooling import GlobalMaxPooling2D
from keras.layers.pooling import GlobalAveragePooling2D
from keras.engine import InputLayer

from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import ImageDataGenerator

# The following cell will test whether the extended version of 
# keras' "load_img" method was successfully installed and is callable.
print('\nload_img tests::')
from keras.preprocessing.image import set_load_img_type
set_load_img_type(0) # prints: 'load_img' calls 'load_img_keras'
set_load_img_type(1) # prints: 'load_img' calls 'load_img_pad'
set_load_img_type(2) # prints: 'load_img' calls 'load_img_pad' (+mask)
set_load_img_type(3) # prints: 'load_img' calls 'load_img_crop'
set_load_img_type(4) # prints: 'load_img' calls 'load_img_multicrop'
im = load_img('test_image.jpg')
ar = im.size[1]/im.size[0]
print('    using none        -> ', im.size, '(aspect ratio = %.3f)' % ar)
im = load_img('test_image.jpg', target_size = (350,290))
ar = im.size[1]/im.size[0]
print('    using (290,350)   -> ', im.size, '(aspect ratio = %.3f)' % ar)
im = load_img('test_image.jpg', target_size = (None,500), interpolation='nearest')
ar = im.size[1]/im.size[0]
print('    using (500,None)  -> ', im.size, '(aspect ratio = %.3f)' % ar)
im = load_img('test_image.jpg', target_size = (500,None))
ar = im.size[1]/im.size[0]
print('    using (None,500)  -> ', im.size, '(aspect ratio = %.3f)' % ar)
im = load_img('test_image.jpg', target_size = (None,100))
ar = im.size[1]/im.size[0]
print('    using (100,None)  -> ', im.size, '(aspect ratio = %.3f)' % ar)
im = load_img('test_image.jpg', target_size = (100,None))
ar = im.size[1]/im.size[0]
print('    using (None,100)  -> ', im.size, '(aspect ratio = %.3f)' % ar)
im = load_img('test_image.jpg', target_size = (None,None))
ar = im.size[1]/im.size[0]
print('    using (None,None) -> ', im.size, '(aspect ratio = %.3f)' % ar)
set_load_img_type(0) # reset to default 'load_img' method

# model architectures
from keras.applications import vgg16
from keras.applications import vgg19
from keras.applications import inception_v3
from keras.applications import xception

# model FCN architectures
from keras.applications import vgg16_fcn
from keras.applications import vgg19_fcn
from keras.applications import inception_v3_fcn
from keras.applications import xception_fcn

# Seed the Random Number Generators
from numpy.random import seed
seed(1)
from tensorflow import set_random_seed
set_random_seed(2)

# Setup parameters
Everything you need to change for your own experiments (parameters, location of train and evaluation datasets) is in the following cell.

In [ ]:
TRAIN_DATA_DIR = 'kagglecatsanddogs_3367a/PetImagesTrain'
VALIDATION_DATA_DIR = 'kagglecatsanddogs_3367a/PetImagesTest'
DATASET_NAME = 'PetImages'

TRAIN_DATA_DIR = 'AE_AVA2_TRAIN'
VALIDATION_DATA_DIR = 'AE_AVA2_TEST'
DATASET_NAME = 'AVA2'

OUTPUT_DIR = 'snapshots'
NO_OF_CLASSES = 2
RATIO_FOR_SMALL_VALIDATION = 0.01
BATCH_SIZE = 8
LR_START = 0.01
LR_MIN = 0.001
FREEZE_UP_TO = 0
DROPOUT_RATE = 0.0

EPOCHS_OF_FULL_EVAL = [5, 10, 15, 20, 30, 40, 45, 50, 55, 60]
EPOCHS_NO = 20

MODEL_NAME = 'v16'  # use 'v16' for VGG16, 'v19' for VGG19
                    # 'x' for Xception and 'i' for Inception V3

In [ ]:
# prepare session
if MODEL_NAME == "v16":
    target_size_ = 224
if MODEL_NAME == "v19":
    target_size_ = 224 
if MODEL_NAME == "i":
    target_size_ = 299
if MODEL_NAME == "x":
    target_size_ = 299

if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

info_string = MODEL_NAME + \
            "-d" + DATASET_NAME + \
            "-b" + str(BATCH_SIZE) + \
            "-d" + str(DROPOUT_RATE)

print('Session info :', info_string)

# Definition of main functions

In [ ]:
def model_exists(str_name):
    for i in os.listdir(OUTPUT_DIR):
        if os.path.isfile(os.path.join(OUTPUT_DIR,i)) and i.startswith(str_name):
            return True
    return False

In [ ]:
def train_model(model, run_id, train_flow, val_flow, force_overwrite=False):
    # Setup a common training  procedure. 
    #
    # For this, we consider two validation sets:
    # 1. A small validation set that uses a RATIO_FOR_SMALL_VALIDATION ratio of the test set
    # 2. A large validation set that uses the whole test set, 
    #    so that the accuracy reported is the test accuracy
    #
    # We then train the model in the following stages:
    # 1. 29 epochs (epochs 1 to 29) using the small validation set
    # 2. 1 epoch (epoch 30) using the large validation set
    # 3. 9 epochs (epochs 31 to 39) using the small validation set
    # 4. 1 epoch (epoch 40) using the large validation set
    # 5. 4 epochs (epochs 41 to 44) using the small validation set
    # 6. 1 epoch (epoch 45) using the large validation set
    # 7. 4 epochs (epochs 41 to 44) using the small validation set
    # 8. 1 epoch (epoch 50) using the large validation set
    # 9. 4 epochs (epochs 51 to 54) using the small validation set
    # 10. 1 epoch (epoch 55) using the large validation set
    # 11. 4 epochs (epochs 56 to 59) using the small validation set
    # 12. 1 epoch (epoch 60) using the large validation set
    #
    # Each stage has its own logger so we can see the course of the whole training procedure,
    # and epochs 30, 40, 45, 50, 55 and 60 report the test accuracy.
    #
    # The reuslting model weights are saved only for epochs where the full validation set is used
    # (i.e. epochs 30, 40, 45, 50, 55 and 60)
    # 
    
    # use the global info string
    global info_string

    if (model_exists(info_string + '_' + run_id + '_')) and not force_overwrite:
        print(" Output for '%s' exists." % (info_string + '_' + run_id + '_'))
        print(" Training skipped.")
        return 
    
    print("    Session %s" % (info_string + '_' + run_id + '_'))

    # Freeze the layers which we don't want to be trained
    for layer in model.layers[:FREEZE_UP_TO]:
        layer.trainable = False
        
    # setup optimizer configuration
    sgd = optimizers.SGD(lr=LR_START, momentum = 0.0)
    
    # compile the model with a SGD/momentum optimizer and a starting learning rate.
    model.compile(loss='categorical_crossentropy', 
                  optimizer=sgd, 
                  metrics=['accuracy'])
    
    # setup model save callback function
    cb_save_model = keras.callbacks.ModelCheckpoint(os.path.join(OUTPUT_DIR,
                    info_string + '_' + run_id + '_' + 'e{epoch:02d}_acc{val_acc:.4f}.hdf5'), 
                    monitor='val_acc', verbose=1, 
                    save_best_only=True, save_weights_only=False, 
                    mode='auto', period=1)
    
    # setup "reduce learning rate" callback function
    cb_reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='acc', factor=0.2, verbose=1,
                                                     patience=3, min_delta=1e-3, 
                                                     cooldown=0, min_lr=LR_MIN)
    
    # set infer steps per epoch
    spe = len(train_flow)
    
    # start training
    print('  ')
    for i in range(1,EPOCHS_NO+1):
        if i in EPOCHS_OF_FULL_EVAL:
            print(' (Epoch %02d - Fine-tuning using all of the evaluation data)' % i)
            print(' -------------------------------------------------------------')
            cb_csv_logger = keras.callbacks.CSVLogger(
                                os.path.join(OUTPUT_DIR,info_string+'_'+run_id+'_training_'+str(i)+'.log'))
            model.fit_generator(train_flow, steps_per_epoch=spe, shuffle=True, 
                                epochs=1, 
                                validation_data=val_flow, 
                                validation_steps=int(val_flow.samples),
                                callbacks = [cb_reduce_lr, cb_save_model, cb_csv_logger])
        else:
            print(' (Epoch %02d - Fine-tuning using a small set of validation data)' % i)
            print(' -----------------------------------------------------------------')
            cb_csv_logger = keras.callbacks.CSVLogger(
                                os.path.join(OUTPUT_DIR,info_string+'_'+run_id+'_training_'+str(i)+'.log'))
            model.fit_generator(train_flow, steps_per_epoch=spe, shuffle=True, 
                                epochs=1, 
                                validation_data=val_flow, 
                                validation_steps=int(RATIO_FOR_SMALL_VALIDATION*val_flow.samples),
                                callbacks = [cb_reduce_lr, cb_save_model, cb_csv_logger])

In [ ]:
def setup_model(fcn=False):
    global target_size_
    if MODEL_NAME == "v16":
        if not fcn:
            print("Model VGG16")
            model = vgg16.VGG16(include_top=False, weights='imagenet', input_shape=[target_size_,target_size_,3])
            x = model.output
            print("    adding flatten layer...") 
            x = layers.Flatten(name='flatten')(x)
            print("    adding FC layer...") 
            x = layers.Dense(4096, activation='relu', name='fc1')(x)
            print("    adding FC layer...") 
            x = layers.Dense(4096, activation='relu', name='fc2')(x)
            if DROPOUT_RATE>0.01:
                print("    adding Dropout layer (rate=.%3f)..." % DROPOUT_RATE) 
                x = layers.Dropout(DROPOUT_RATE)(x)
            print("    adding FC layer for %d classes..." % NO_OF_CLASSES) 
            x = layers.Dense(NO_OF_CLASSES, activation='softmax', name='predictions')(x)
            model = Model(model.input, x, name='vgg16')
            print("    compiled 'VGG16 model.") 
        else:
            print("Model VGG16 FCN")
            model = vgg16_fcn.VGG16_fcn(weights='imagenet', input_shape=[None,None,3], classes=NO_OF_CLASSES)
            x = model.output
            if DROPOUT_RATE>0.01:
                print("    adding Dropout layer (rate=.%3f)..." % DROPOUT_RATE) 
                x = keras.layers.Dropout(DROPOUT_RATE)(x)
            print("    adding GlobalMaxPooling2d layer...")
            x = GlobalMaxPooling2D()(x)
            model = Model(model.input, x, name='VGG16_FCN')
            print("    compiled 'VGG16_FCN' model.")
    if MODEL_NAME == "v19":
        if not fcn:
            print("Model VGG19")
            model = vgg19.VGG19(include_top=False, weights='imagenet',input_shape=[target_size_,target_size_,3])
            x = model.output
            print("    adding flatten layer...") 
            x = layers.Flatten(name='flatten')(x)
            print("    adding FC layer...") 
            x = layers.Dense(4096, activation='relu', name='fc1')(x)
            print("    adding FC layer...") 
            x = layers.Dense(4096, activation='relu', name='fc2')(x)
            if DROPOUT_RATE>0.01:
                print("    adding Dropout layer (rate=.%3f)..." % DROPOUT_RATE) 
                x = layers.Dropout(DROPOUT_RATE)(x)
            print("    adding FC layer for %d classes..." % NO_OF_CLASSES) 
            x = layers.Dense(NO_OF_CLASSES, activation='softmax', name='predictions')(x)
            model = Model(model.input, x, name='vgg19')
            print("    compiled 'VGG19 model.") 
        else:
            print("Model VGG19 FCN")
            model = vgg19_fcn.VGG19_fcn(weights='imagenet', input_shape=[None,None,3], classes=NO_OF_CLASSES)
            x = model.output
            if DROPOUT_RATE>0.01:
                print("    adding Dropout layer (rate=.%3f)..." % DROPOUT_RATE) 
                x = keras.layers.Dropout(DROPOUT_RATE)(x)
            print("    adding GlobalMaxPooling2d layer...")
            x = GlobalMaxPooling2D()(x)
            model = Model(model.input, x, name='VGG19_FCN')
            print("    compiled 'VGG19_FCN' model.")

    if MODEL_NAME == "i":
        if not fcn:
            print("Model InceptionV3")
            model = inception_v3.InceptionV3(include_top=False, weights='imagenet', classes=NO_OF_CLASSES)
            x = model.output
            print("    adding GlobalMaxPooling2d layer...")
            x = layers.GlobalAveragePooling2D(name='avg_pool')(x)
            if DROPOUT_RATE>0.01:
                print("    adding Dropout layer (rate=.%3f)..." % DROPOUT_RATE) 
                x = layers.Dropout(DROPOUT_RATE)(x)
            print("    adding FC layer for %d classes..." % NO_OF_CLASSES) 
            x = layers.Dense(classes, activation='softmax', name='predictions')(x)
            model = Model(model.input, x, name='InceptionV3')
            print("    compiled 'InceptionV3 model.") 
        else:
            print("Model InceptionV3 FCN")
            model = inception_v3_fcn.InceptionV3_fcn(weights='imagenet', classes=NO_OF_CLASSES)
            x = model.output
            if DROPOUT_RATE>0.01:
                print("    adding Dropout layer (rate=.%3f)..." % DROPOUT_RATE) 
                x = keras.layers.Dropout(DROPOUT_RATE)(x)
            print("    adding GlobalMaxPooling2d layer...")
            x = GlobalMaxPooling2D()(x)
            model = Model(model.input, x, name='InceptionV3_FCN')
            print("    compiled 'InceptionV3_FCN' model.")
    if MODEL_NAME == "x":
        if not fcn:
            print("Model Xception")
            model = xception.Xception(include_top=False, weights='imagenet', classes=NO_OF_CLASSES)
            x = model.output
            print("    adding GlobalMaxPooling2d layer...")
            x = layers.GlobalAveragePooling2D(name='avg_pool')(x)
            if DROPOUT_RATE>0.01:
                print("    adding Dropout layer (rate=.%3f)..." % DROPOUT_RATE) 
                x = layers.Dropout(DROPOUT_RATE)(x)
            print("    adding FC layer for %d classes..." % NO_OF_CLASSES) 
            x = layers.Dense(NO_OF_CLASSES, activation='softmax', name='predictions')(x)
            model = Model(model.input, x, name='Xception')
            print("    compiled 'Xception model.") 
        else:
            print("Model Xception FCN")
            model = xception_fcn.Xception_fcn(weights='imagenet', classes=NO_OF_CLASSES)
            model = inception_v3_fcn.InceptionV3_fcn(weights='imagenet', classes=NO_OF_CLASSES)
            x = model.output
            if DROPOUT_RATE>0.01:
                print("    adding Dropout layer (rate=.%3f)..." % DROPOUT_RATE) 
                x = keras.layers.Dropout(DROPOUT_RATE)(x)
            print("    adding GlobalMaxPooling2d layer...")
            x = GlobalMaxPooling2D()(x)
            model = Model(model.input, x, name='Xception_FCN')
            print("    compiled 'Xception_FCN' model.")
    return model

In [ ]:
def setup_data_flows(upscale_ratio=1.0):
    global target_size_
    
    # prepare data generators
    print("Setting up train data generators...")
    train_data = ImageDataGenerator(rescale=1./255, horizontal_flip=True)
    train_flow = train_data.flow_from_directory(
                 TRAIN_DATA_DIR, target_size=(int(upscale_ratio*target_size_), int(upscale_ratio*target_size_)),
                 batch_size=BATCH_SIZE, class_mode='categorical')

    print("Setting up test data generators...")
    test_data = ImageDataGenerator(rescale=1./255)
    test_flow = test_data.flow_from_directory(
                VALIDATION_DATA_DIR, target_size=(int(upscale_ratio*target_size_), int(upscale_ratio*target_size_)), 
                batch_size=BATCH_SIZE, class_mode='categorical')

    return train_flow, test_flow

# Experiments

In this section the following experiments are performed:
* Experiment #1 - Finetune the original model.
* Experiment #2 - Create an FCN version and finetune using the original input size.  
* Experiment #3 - Finetune the FCN model using 1.5x of the original input size.  
* Experiment #4 - Finetune the FCN model using 2.0x of the original input size.  
* Experiment #5 - Finetune the FCN model using 3.0x of the original input size.  
* Experiment #6 - Finetune the FCN model using 1.5x of the original input size (cropping method). 
* Experiment #7 - Finetune the FCN model using 1.5x of the original input size (padding method). 
* Experiment #8 - Finetune the FCN model using 1.5x of the original input size (multi-crop method).
* Experiment #9 - Finetune the FCN model using 1.5x of the original input size (multi-crop method), also introducing a skip connection.   

### Test #1 - Finetune the original model.

In [ ]:
set_load_img_type(0) 
model = setup_model()
train_flow, test_flow = setup_data_flows(upscale_ratio=1.0)
train_model(model, 'orig', train_flow, test_flow)

In [ ]:
# delete original model from memory
del model
gc.collect()

### Test #2 - Finetune the FCN version using the original input size.

In [ ]:
set_load_img_type(0)
model_fcn = setup_model(fcn=True)
train_flow, test_flow = setup_data_flows(upscale_ratio=1.0)
train_model(model_fcn, 'fcn_10', train_flow, test_flow)

### Test #3 - Finetune the FCN model using 1.5x of the original input size.

In [ ]:
set_load_img_type(0)
model_fcn = setup_model(fcn=True)
train_flow_15, test_flow_15 = setup_data_flows(upscale_ratio=1.5)
train_model(model_fcn, 'fcn_15', train_flow_15, test_flow_15)

### Test #4 - Finetune the FCN model using 2.0x of the original input size.

In [ ]:
set_load_img_type(0)
model_fcn = setup_model(fcn=True)
train_flow_20, test_flow_20 = setup_data_flows(upscale_ratio=2.0)
train_model(model_fcn, 'fcn_20', train_flow_20, test_flow_20)

### Test #5 - Finetune the FCN model using 3.0x of the original input size.

In [ ]:
set_load_img_type(0)
model_fcn = setup_model(fcn=True)
train_flow_30, test_flow_30 = setup_data_flows(upscale_ratio=3.0)
train_model(model_fcn, 'fcn_30', train_flow_30, test_flow_30) # Not completed on Xception

### Test #6 - Finetune the FCN model using 1.5x of the original input size (padding method).

In [ ]:
set_load_img_type(1) # 'load_img' now calls 'load_img_pad'
model_fcn = setup_model(fcn=True)
train_flow_15, test_flow_15 = setup_data_flows(upscale_ratio=1.5)
train_model(model_fcn, 'fcn_15_pad', train_flow_15, test_flow_15)

### Test #7 - Finetune the FCN model using 1.5x of the original input size (crop method).

In [ ]:
set_load_img_type(3) # 'load_img' now calls 'load_img_crop'
model_fcn = setup_model(fcn=True)
train_flow_15, test_flow_15 = setup_data_flows(upscale_ratio=1.5)
train_model(model_fcn, 'fcn_15_crop', train_flow_15, test_flow_15)

### Test #8 - Finetune the FCN model using 1.5x of the original input size (multi-crop method).

In [ ]:
set_load_img_type(4) # 'load_img' now calls 'load_img_multicrop'
model_fcn = setup_model(fcn=True)
train_flow_15, test_flow_15 = setup_data_flows(upscale_ratio=1.5)
train_model(model_fcn, 'fcn_15_multicrop', train_flow_15, test_flow_15)

### Test #9 - Create a copy of the FCN model with a skip conncetion and finetune the new  model using 1.5x of the original input size (multi-crop method).

In [ ]:
print('~~~ Setting up skip connection model ~~~')
# ONLY implemented for VGG16

# copy model
print('    copying model...\n')
model_fcn = setup_model(fcn=True)
model_fcn_skip = keras.models.clone_model(model_fcn)
print(model_fcn.summary(line_length=112))

# remove GlobalMaxPooling, predictions, fc2 and fc1)
print('\nRemoving last layer...')
model_fcn_skip.layers.pop()
model_fcn_skip.layers.pop()
model_fcn_skip.layers.pop()
model_fcn_skip.layers.pop()

# add skip connnection
print('\nAdding skip connection...')
b2_pool = model_fcn_skip.get_layer("block2_pool").output
b5_pool = model_fcn_skip.get_layer("block5_pool").output
print('    Adding Conv2d after "block2_pool" ("skip_1")...')
x = keras.layers.Conv2D(128,(1,1),activation='relu',name='skip_1',padding='same')(b2_pool)
print('    Adding MaxPooling2D after "block2_pool" ("skip_2")...')
x = keras.layers.MaxPooling2D((65,65),strides=(2,2),name='skip_2')(x)
print('    Adding Concatenate...')
x = keras.layers.Concatenate(name='concat')([x, b5_pool])
print('    Adding "fc1"...')
x = keras.layers.Conv2D(4096,(7, 7),strides=(1,1),activation='relu',name='fc1',padding='valid')(x)
print('    Adding "fc2"...')
x = keras.layers.Conv2D(4096,(1, 1),strides=(1,1),activation='relu',name='fc2',padding='valid')(x)
print('    Adding "prediction"...')
x = keras.layers.Conv2D(2,(1, 1),strides=(1,1),activation='softmax',name='predictions',padding='valid')(x)

# add dropout layer
if DROPOUT_RATE>0.01:
    print('    Adding Dropout (rate=.%3f) on FCN...' % DROPOUT_RATE) 
    x = keras.layers.Dropout(DROPOUT_RATE)(x)

# add global max pooling layer
print('    Adding GlobalMaxPooling2d...')
x = GlobalMaxPooling2D()(x)

# compile model
model_fcn_skip = Model(model_fcn_skip.input, x, name='GlobalMaxPooledFCNmulticrop')
print('    ~~~ Created new model: model_gmax15_2_multicrop (', model_fcn_skip.name, ')')

print('\nModel summary with skip connection...')
print(model_fcn_skip.summary(line_length=112))
print("\n")

# reload weights
model_fcn.save_weights('tmp_w')
model_fcn_skip.load_weights('tmp_w', by_name=True, reshape=False, skip_mismatch=True)

In [ ]:
set_load_img_type(4)  # 'load_img' now calls 'load_img_multicrop'
train_flow_15, test_flow_15 = setup_data_flows(upscale_ratio=1.5)
train_model(model_fcn_skip, 'fcn_15_multicrop_skip', train_flow_15, test_flow_15)